In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df
from db_config import db_engine
from db_utils import sp2df, get_columns, df2csv, csv2db_stage, csv2db, csvs2db

In [3]:
from asteroid_element import get_ast_ref_elts, add_asteroid_elts, update_asteroid_elements
from asteroid_element_synch import process_dates

In [4]:
# Typing
from typing import Tuple, List

In [5]:
epoch_out = 59000
elt_dates = sp2df(sp_name='KS.GetAsteroidRefElementDates', params={'epoch':epoch_out})

In [6]:
elt_dates

,TimeID,epoch,AsteroidCount
0,60416640,41956.0,1
1,62537760,43429.0,1
2,63433440,44051.0,1
3,63678240,44221.0,1
4,64319040,44666.0,1
...,...,...,...
5242,84975840,59011.0,2
5243,84977280,59012.0,1
5244,84978720,59013.0,2
5245,84980160,59014.0,6


In [7]:
# First epoch
epoch0 = int(elt_dates.epoch[0])

In [8]:
# Start with a base simulation of the Sun, Planets and Moon
sim = make_sim_planets(epoch=epoch0)
# Set simulation time t to match epoch
sim.t = sim.epoch
# Set the number of active particles to the base simulation
# https://rebound.readthedocs.io/en/latest/ipython/Testparticles.html
sim.N_active = sim.N

Added bodies to simulation.


In [9]:
# Get the elements
# elts = sp2df(sp_name='KS.GetAsteroidRefElements', params={'epoch':epoch0})
elts = get_ast_ref_elts(epoch0)

In [10]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,60416640,2060 T-2,41956.0,2.373136,0.202053,0.012784,3.08049,3.456215,6.158491,6.202047


In [11]:
# First set of asteroid IDs
asteroid_ids = add_asteroid_elts(sim=sim, elts=elts)

In [12]:
asteroid_ids

array([1413593])

In [13]:
epoch = epoch0 + 100
sim.integrate(epoch, exact_finish_time=1)

In [14]:
sim.particles[-1]

<rebound.Particle object, m=0.0 x=1.2967077821137172 y=1.454334678850183 z=-0.019483520340996357 vx=-0.009079326348883594 vy=0.009824672557089268 vz=-0.00011830502105655515>

In [15]:
# Copy of elements
elts_old = elts.copy()

In [16]:
update_asteroid_elements(sim=sim, elts=elts, epoch=epoch)

In [17]:
elts_old

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,60416640,2060 T-2,41956.0,2.373136,0.202053,0.012784,3.08049,3.456215,6.158491,6.202047


In [18]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,60560640,2060 T-2,42056.0,2.367485,0.200858,0.012753,3.083035,-2.838647,0.598284,0.397882


In [19]:
elts_new =  get_ast_ref_elts(elt_dates.epoch[1])

In [20]:
elts_new

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413594,62537760,2678 T-3,43429.0,2.260404,0.258348,0.168633,3.569416,2.591761,0.46277,0.267387


In [21]:
pd.concat([elts, elts_new]).reset_index(drop=True)

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,60560640,2060 T-2,42056.0,2.367485,0.200858,0.012753,3.083035,-2.838647,0.598284,0.397882
1,1413594,62537760,2678 T-3,43429.0,2.260404,0.258348,0.168633,3.569416,2.591761,0.462770,0.267387


In [15]:
def process_dates(epoch):
    """Process all the orbital elements to the reference epoch"""

    # All the reference dates with orbital elements that must be brought forward
    elt_dates = sp2df(sp_name='KS.GetAsteroidRefElementDates', params={'epoch':epoch})
    # The first date to process
    epoch = np.int32(elt_dates.epoch[0])
    # Get the elements on the first date
    elts = get_ast_ref_elts(epoch=epoch)

    # Number of dates to process
    epochs: np.array = elt_dates.epoch.values
    N_date: int = epochs.shape[0]

    # Iterate through the remaining dates
    # for i in range(1, N_date):
    for i in range(2):
        # Two epochs: current one (with quoted elements) and next one
        epoch: int = epochs[i]
        epoch_next: int = epochs[i+1]

        # Initialize a planets simulation as of epoch
        sim = make_sim_planets(epoch=epoch, load_file=True)
        sim.t = np.float64(epoch)
        sim.N_active = sim.N
        # Add asteroids previously processed with their current orbital elements
        asteroid_ids = add_asteroid_elts(sim=sim, elts=elts)

        # Integrate to the next epoch
        sim.integrate(epoch_next, exact_finish_time=1)
        # Update the asteroid elements as of this epoch
        update_asteroid_elements(sim=sim, elts=elts, epoch=epoch)
        # Fetch the new orbital elements at the next epoch
        elts_new = get_ast_ref_elts(epoch=epoch_next)
        # Append the new elements to the main elts frame
        elts = pd.concat([elts, elts_new]).reset_index(drop=True)

    return sim, elts

In [16]:
sim, elts = process_dates(59000)

In [17]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,f,M
0,1413593,62537760,2060 T-2,43429.0,2.347232,0.201381,0.012812,3.082599,-2.823637,3.396120,-2.768567
1,1413593,62537760,2060 T-2,43429.0,2.358819,0.207335,0.012790,3.085074,-2.835494,2.943949,2.848092
2,1413594,62537760,2678 T-3,43429.0,2.260404,0.258348,0.168633,3.569416,2.591761,0.462770,0.267387
